# **HOMEWORK 1 - Regressione Lineare**

In questo homework dovrete:

1. Scrivere una funzione di pipeline che deve gestire l' allenamento di un modello di regressione lineare al variare degli iperparametri forniti. Nello specifico:
    * Deve applicare la PCA, se presente.
    
    * Deve applicare la standardizzazione, se presente.

    * Deve applicare la regolarizzazione, se presente.

    * Deve allenare il modello di regressione lineare.

    * Deve calcolare la MAE.

2. Scrivere una funzione che utilizzi la `pipeline` definita al punto 1 e che testi tutte le configurazioni possibili presenti in `configs`. Nel dettaglio la funzione deve:
    * Dividere il dataset in train e validation.

    * Calcolare, grazie alla funzione `pipeline` definita al punto 1, quale configurazione ottiene il punteggio migliore (quale configurazione ha la MAE di validation più bassa).

3. Scrivere una funzione che utilizzi la configurazione migliore prodotta dalla funzione definita al punto 2 e la testi sul test set. 

4. Stampare:
    * La migliore configurazione

    * Il miglior MAE di validation 

    * Il migliore MAE di train

    * Il MAE di test 


Il codice che di seguito trovate già fornito deve essere utilizzato per la risoluzione dell' homework, **NON MODIFICATELO IN ALCUN MODO**.

## **Dataset Wine Quality White**

Il dataset da utilizzare è `wine-quality-white` della libreria `scikit-learn`. Il dataset contiene 11 variabili numeriche + 1 di target che classifica il vino in diverse categorie di qualità. Per il nostro obiettivo la variabile di target è considerata come `float`, permettendoci di applicare la regressione lineare. All' interno del dataset sono contenuti 4898 campioni. 

In [58]:
# Questa cella contiene tutte le librerie di cui necessitate per risolvere l' homework.
# Ricordate di eseguirla prima di iniziare.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.datasets import fetch_openml
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler

In [59]:
hyperparams = {
    # PCA
    'use_pca': [True, False],
    'pca_standardize': [True, False],
    'pca_components': [3, 5, 10],
    # Data standardization
    'data_standardize': [True, False],
    # Regularization l2
    'use_regularization': [True, False],
    'reg_lambda': [0.1, 1, 10],
}

# Calcoliamo tutte le possibili combinazioni di iperparametri
import itertools
combinations = list(itertools.product(*hyperparams.values()))
configs = [dict(zip(hyperparams.keys(), combination)) for combination in combinations]

# Evitiamo le combinazioni non valide
for config in configs:
    if not config['use_pca']:
        config['pca_standardize'] = None
        config['pca_components'] = None
    if not config['use_regularization']:
        config['reg_lambda'] = None
configs = set([tuple(config.items()) for config in configs])

# Convertiamo di nuovo in lista di dizionari
configs = [dict(config) for config in configs]
print(f'Numero di combinazioni: {len(configs)}')

Numero di combinazioni: 56


In `configs` avete una lista di dizionari, ogni dizionario contiene una possibile combinazione di hyperparametri da utilizzare nella fase di training. 

In [60]:
# Carica il dataset Wine Quality White
data = fetch_openml(name='wine-quality-white', version=1, as_frame=True)
X = data.data
y = data.target.astype(float)  # Assicura che il target sia float per la regressione

def pipeline(X_train, y_train, X_val, y_val, hyperparams):
    """
    Addestra un modello di regressione lineare con eventuale PCA e regolarizzazione L2.
    """
    if hyperparams['use_pca']:
        if hyperparams['pca_standardize']:
            scaler_pca = StandardScaler().fit(X_train)
            X_train_pca = scaler_pca.transform(X_train)
            X_val_pca = scaler_pca.transform(X_val)
        else:
            X_train_pca, X_val_pca = X_train, X_val
            
        pca = PCA(n_components=hyperparams['pca_components']).fit(X_train_pca)
        X_train, X_val = pca.transform(X_train_pca), pca.transform(X_val_pca)

    if hyperparams['data_standardize']:
        scaler_data = StandardScaler().fit(X_train)
        X_train, X_val = scaler_data.transform(X_train), scaler_data.transform(X_val)

    # Aggiunge il termine costante ai dati
    X_train = np.concatenate([np.ones((len(X_train), 1)), X_train], axis=1)
    X_val = np.concatenate([np.ones((len(X_val), 1)), X_val], axis=1)

    XtX = X_train.T @ X_train
    Xty = X_train.T @ y_train

    # Calcolo della soluzione di regressione lineare
    if hyperparams['use_regularization']:
        I = np.eye(X_train.shape[1])
        XtX_reg = XtX + hyperparams['reg_lambda'] * I
        parametri = np.linalg.solve(XtX_reg, Xty)
    else:
        parametri = np.linalg.solve(XtX, Xty) if X_train.shape[0] >= X_train.shape[1] else np.linalg.lstsq(XtX, Xty, rcond=None)[0]
       

    # Calcolo predizione
    def calc_mae(X, y):
        return np.abs(X.dot(parametri) - y).mean()

    # Calcola il MAE
    mae_train = calc_mae(X_train, y_train)
    mae_val = calc_mae(X_val, y_val)

    return mae_train, mae_val

def try_pipeline(X, y, hyperparams):
    # Configurazione divisione dataset
    train_ratio = 0.8  # 80% training+validation, 20% test
# Dividi il dataset in training e test set
    split_point = int(train_ratio * len(X))
    X_train_val, X_test = np.split(X, [split_point])
    y_train_val, y_test = np.split(y, [split_point])

    print(f"Dataset diviso - Train+Val: {X_train_val.shape}, Test: {X_test.shape}")

# Dividi il training set in training set effettivo e validation set
    val_split = int(train_ratio * len(X_train_val))
    X_train, X_val = X_train_val[:val_split], X_train_val[val_split:]
    y_train, y_val = y_train_val[:val_split], y_train_val[val_split:]

    print("\nDimensioni subset finali:")
    print(f"Training: {X_train.shape}, Validation: {X_val.shape}, Test: {X_test.shape}")
    total_samples = len(X_train) + len(X_val) + len(X_test)
    print(f"Controllo consistenza: {total_samples} samples totali")

# Trova la configurazione di iperparametri migliore
    top_mae_val=float(10000)    #inizializzazione ad un valore alto cosi sono sicuro che venga trovato un mae inferiore
    top_mae_train=0  
    top_config=  None  
    print("\n\nCERCO MIGLIORE CONFIGURAZIONE IPERPARAMETRI:")
    for config in hyperparams: #itero per ogni configurazione
        print("Provo configurazione :",config)


# Riallena il modello sul training set completo
        X_train_copy = X_train.copy()
        X_val_copy = X_val.copy()
# Calcola il MAE sul test set
        mae_train, mae_val = pipeline(X_train_copy, y_train, X_val_copy, y_val, config)
        print("MAE di Train: ",mae_train)
        print("MAE di Val:",mae_val)

    if mae_val < top_mae_val:
            top_mae_val = mae_val
            top_mae_train = mae_train
            top_config = config

    print("\n\n--------Migliore configurazione trovata---------")
    print(top_config)

    return top_config, top_mae_train, top_mae_val

# Stampa  risultati
def pipeline_best_mae(X, y, hyperparams):
    # Configurazione split iniziale
    TRAIN_SIZE = 0.8  # 80% training, 20% test
    
    # Divisione dataset con calcolo efficiente
    split_idx = int(len(X) * TRAIN_SIZE)
    X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
    y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

# Ricerca migliori iperparametri
    top_config, top_mae_train,top_mae_val = try_pipeline(X, y, hyperparams)
    
# Valutazione prestazioni finali
    mae_train,final_test_mae = pipeline(
    X_train.copy(), y_train.copy(),
    X_test.copy(), y_test.copy(),
    top_config
    )

    return top_config, top_mae_train, top_mae_val,final_test_mae


# Esecuzione e visualizzazione risultati
config, mae_train,mae_val,final_test_mae = pipeline_best_mae(X, y, configs)

print("\n\n=== RISULTATI FINALI ===")
print(f"Configurazione ottimale:\n{ {k:v for k,v in config.items() if v is not None} }")
print(f"MAE Training: {mae_train:.5f}")
print(f"MAE Validation: {mae_val:.5f}") 
print(f"MAE Test: {final_test_mae:.5f}")

Dataset diviso - Train+Val: (3918, 11), Test: (980, 11)

Dimensioni subset finali:
Training: (3134, 11), Validation: (784, 11), Test: (980, 11)
Controllo consistenza: 4898 samples totali


CERCO MIGLIORE CONFIGURAZIONE IPERPARAMETRI:
Provo configurazione : {'use_pca': True, 'pca_standardize': True, 'pca_components': 10, 'data_standardize': True, 'use_regularization': True, 'reg_lambda': 1}
MAE di Train:  0.6042886346353137
MAE di Val: 0.5749848230411283
Provo configurazione : {'use_pca': False, 'pca_standardize': None, 'pca_components': None, 'data_standardize': False, 'use_regularization': True, 'reg_lambda': 0.1}
MAE di Train:  0.6059380654544342
MAE di Val: 0.5737295673097799
Provo configurazione : {'use_pca': True, 'pca_standardize': True, 'pca_components': 10, 'data_standardize': True, 'use_regularization': True, 'reg_lambda': 10}
MAE di Train:  0.6043817772396395
MAE di Val: 0.5730065241180352
Provo configurazione : {'use_pca': True, 'pca_standardize': False, 'pca_components': 5,

/usr/local/lib/python3.11/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(
/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


MAE di Train:  0.6913890082178289
MAE di Val: 0.6577989723324451
Provo configurazione : {'use_pca': True, 'pca_standardize': True, 'pca_components': 5, 'data_standardize': False, 'use_regularization': False, 'reg_lambda': None}
MAE di Train:  0.6437946531482523
MAE di Val: 0.6122602649455579
Provo configurazione : {'use_pca': True, 'pca_standardize': False, 'pca_components': 3, 'data_standardize': True, 'use_regularization': True, 'reg_lambda': 10}
MAE di Train:  0.6922572695921927
MAE di Val: 0.6592176897080271
Provo configurazione : {'use_pca': True, 'pca_standardize': True, 'pca_components': 3, 'data_standardize': False, 'use_regularization': True, 'reg_lambda': 0.1}
MAE di Train:  0.6530127072529314
MAE di Val: 0.637226737975966
Provo configurazione : {'use_pca': True, 'pca_standardize': True, 'pca_components': 5, 'data_standardize': True, 'use_regularization': False, 'reg_lambda': None}
MAE di Train:  0.6437946531482523
MAE di Val: 0.6122602649455579
Provo configurazione : {'use_p